In [2]:
%load_ext autoreload

%autoreload 2
import imp
import factorilog


In [3]:
%pdb 

Automatic pdb calling has been turned ON


In [83]:


from factorilog import Wire, WireColor

with open("blueprints/sample.bp","r") as file:
    sample = file.read()
    c = factorilog.Layout.fromBlueprint(sample)


In [84]:
netlist_meta = c.toNetString(meta=True)
print(netlist_meta)

d: medium-electric-pole        | 1
c <= d: 1 black if green > 2   | 2
e <= d: red = green + 0        | 3
b <= a, c: @ all if black = 0  | 4
b <= c, e: @ red if black = 1  | 5
b: medium-electric-pole        | 6
a <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
a | red 4i-7
b | green 5o-6 4o-6
c | green 2o-5i 4i-5i
d | red 1-3i 2i-1
e | red 3o-5i


In [85]:
layout = factorilog.Layout.fromNetlist(netlist_meta)

{
  "Entities": [
    {
      "Descriptor": {
        "name": "medium-electric-pole"
      },
      "ID": 1,
      "nets": {
        "pass": [
          "d"
        ],
        "out": null
      }
    },
    {
      "Descriptor": {
        "behavior": {
          "decider_conditions": {
            "copy_count_from_input": false,
            "output_signal": {
              "type": "virtual",
              "name": "signal-black"
            },
            "comparator": ">",
            "constant": 2,
            "first_signal": {
              "type": "virtual",
              "name": "signal-green"
            }
          }
        },
        "name": "decider-combinator"
      },
      "ID": 2,
      "nets": {
        "in": [
          "d"
        ],
        "out": [
          "c"
        ]
      }
    },
    {
      "Descriptor": {
        "behavior": {
          "arithmetic_conditions": {
            "first_signal": {
              "type": "virtual",
              "name": "signal-gree

In [86]:
print(layout.toNetString(meta=True))

d: medium-electric-pole        | 1
c <= d: 1 black if green > 2   | 2
e <= d: red = green + 0        | 3
b <= c, a: @ all if black = 0  | 4
b <= c, e: @ red if black = 1  | 5
b: medium-electric-pole        | 6
a <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
b | green 6-5o 4o-6
c | green 5i-2o 5i-4i
d | red 2i-1 1-3i
a | red 4i-7
e | red 5i-3o


In [87]:
from slpp import slpp as lua
bp=lua.encode(layout.exportBlueprint())
#print(bp.replace("  "," ").replace("\t","").replace("\n",""))
print(bp.replace("  "," ").replace("\t","  "))

{
  {
    ["connections"] =   {
      ["1"] =     {
        ["red"] =       {
                  {
            ["entity_id"] = 2,
            ["circuit_id"] = 1
          },
                  {
            ["entity_id"] = 3,
            ["circuit_id"] = 1
          }
        }
      }
    },
    ["position"] =   {
      ["y"] = -2,
      ["x"] = -5
    },
    ["entity_number"] = 1,
    ["name"] = "medium-electric-pole"
  },
  {
    ["position"] =   {
      ["y"] = -2,
      ["x"] = -2.5
    },
    ["entity_number"] = 2,
    ["control_behavior"] =   {
      ["decider_conditions"] =     {
        ["copy_count_from_input"] = false,
        ["output_signal"] =       {
          ["type"] = "virtual",
          ["name"] = "signal-black"
        },
        ["comparator"] = ">",
        ["constant"] = 2,
        ["first_signal"] =       {
          ["type"] = "virtual",
          ["name"] = "signal-green"
        }
      }
    },
    ["connections"] =   {
      ["1"] =     {
        ["red"] =  

In [88]:
a=lua.decode('{["c"] = 5}')
b=lua.decode('{[5] = 5}')
print(a==b)
print(lua.encode(a))
print(lua.encode(b))

False
{
	["c"] = 5
}
{
5
}


In [89]:
from enum import Enum
class test(Enum):
    a=4
    b=3
test(3)==test(3)

True

In [80]:
"%s" % "4"

'4'

In [10]:
#wires per term
l_ents = list(c.entities)
for ent in l_ents:
    print(ent.name,len(ent.terminals))
    for term in ent.terminals:
        print("\t",len(term.wires))
        

medium-electric-pole 1
	 2
decider-combinator 2
	 2
	 1
medium-electric-pole 1
	 2
decider-combinator 2
	 1
	 1
constant-combinator 1
	 1
decider-combinator 2
	 3
	 1
arithmetic-combinator 2
	 1
	 1


In [20]:
ar = l_ents[1]
con = c.getConnectedTerminals(ar.terminals[0],WireColor.red)
print(con)
print(list(con)[0].ent.name)


{<factorilog.Terminal object at 0x7f2a64029710>, <factorilog.Terminal object at 0x7f2a64029550>}
constant-combinator


In [21]:
net = c.getNetlist()
print(len(net))
for hyp in list(net):
    print(hyp.color, len(hyp.terminals))

5
WireColor.red 3
WireColor.green 3
WireColor.green 3
WireColor.red 2
WireColor.red 2


In [22]:
w=l_ents[1].terminals[0].wires
for w1 in l_ents[1].terminals[0].wires:
    print(w1.color)
    print(w1.terminals)

WireColor.green
frozenset({<factorilog.Terminal object at 0x7f2a64029550>, <factorilog.Terminal object at 0x7f2a640295f8>})
WireColor.red
frozenset({<factorilog.Terminal object at 0x7f2a64029710>, <factorilog.Terminal object at 0x7f2a64029550>})


In [23]:
#num wires
wires = set()
for ent in list(c.ents):
    for term in ent.terminals:
        wires.update(term.wires)
print(len(wires))

8


In [26]:

import pprint
from slpp import slpp as lua
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(lua.decode(sample))

[ { 'connections': { '1': { 'red': [ {'circuit_id': 1, 'entity_id': 2},
                                     {'circuit_id': 1, 'entity_id': 3}]}},
    'entity_number': 1,
    'name': 'medium-electric-pole',
    'position': {'x': -5, 'y': -2}},
  { 'connections': { '1': {'red': [{'entity_id': 1}]},
                     '2': {'green': [{'circuit_id': 1, 'entity_id': 5}]}},
    'control_behavior': { 'decider_conditions': { 'comparator': '>',
                                                  'constant': 2,
                                                  'copy_count_from_input': False,
                                                  'first_signal': { 'name': 'signal-green',
                                                                    'type': 'virtual'},
                                                  'output_signal': { 'name': 'signal-black',
                                                                     'type': 'virtual'}}},
    'direction': 2,
    'entity_number': 2,
  

In [284]:
import test_parser
import json
imp.reload(test_parser)

text=\
"""84"""

parser = test_parser.TestParser(parseinfo=False)
ast = parser.parse(text, rule_name='start')
print(ast)
print(json.dumps(ast, indent=2)) # ASTs are JSON-friendy

['84']
[
  "84"
]
